In [1]:
import requests

def search_searxng(query, instance_url="http://localhost:32768"):
    response = requests.post(
        f"{instance_url}/search",
        data={
            "q": query,
            "categories": "general",
            "language": "en",
            "format": "json"
        },
        timeout=10
    ) 
    
    if response.ok:
        results = response.json().get("results", [])
        return [(r["title"], r["url"]) for r in results]
    else:
        raise Exception(f"Search failed: {response.status_code} - {response.text}")

# Example usage
results = search_searxng("A good phone repair business")
for title, url in results:
    print(f"{title}: {url}")

ConnectionError: HTTPConnectionPool(host='localhost', port=32768): Max retries exceeded with url: /search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x72a1e8ce2980>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [57]:
import requests
from bs4 import BeautifulSoup

SEARX_INSTANCE = "http://localhost:32768"
DEFAULT_HEADERS = {"User-Agent": "Mozilla/5.0"}

def search_localsearch_via_searx(query: str, location: str = "brisbane", max_results: int = 5):
    search_query = f"{query} site:localsearch.com.au {location}"

    params = {
        "q": search_query,
        "format": "json",
        "language": "en",
        "safesearch": "0",
        "engines": "startpage,brave,mojeek,qwant"
    }

    print(f"[1] Searching via SearXNG: '{search_query}'")
    res = requests.get(f"{SEARX_INSTANCE}/search", params=params, headers=DEFAULT_HEADERS, timeout=10)
    res.raise_for_status()

    results = res.json().get("results", [])
    filtered = [r for r in results if "localsearch.com.au/profile/" in r.get("url", "")]
    
    print(f"[2] Found {len(filtered)} localsearch profile links")
    return [(r["title"], r["url"]) for r in filtered[:max_results]]


if __name__ == "__main__":
    query = "pc repair shop"
    results = search_localsearch_via_searx(query)

    print("[3] Top businesses from SearXNG:")
    for name, url in results:
        print(f"- {name}\n  {url}\n")



[1] Searching via SearXNG: 'pc repair shop site:localsearch.com.au brisbane'
[2] Found 1 localsearch profile links
[3] Top businesses from SearXNG:
- ZYLAX COMPUTERS SYDNEY in Shop 5/189 Kent Street Sydney 2000 NSW | Localsearch
  https://www.localsearch.com.au/profile/zylax-computers/clwzugbhl000n08k0apbkhkzc



In [1]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import gradio as gr
import torch

import requests
from bs4 import BeautifulSoup

import pandas as pd
from tabulate import tabulate
import textwrap
import re

/home/joshua/miniconda3/envs/llm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_path = "/mnt/models/llm_storage/Llama-3.2-3B-Instruct"
#model_path = "/home/joshua/llms/llama3.1_PC_Build/Llama-3.1-8B-Instruct"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="cuda:0"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


: 

In [4]:
def format_messages(messages):
    prompt = ""
    for msg in messages:
        role = msg["role"]
        if role == "system":
            prompt += f"<|system|>\n{msg['content']}\n"
        elif role == "user":
            prompt += f"<|user|>\n{msg['content']}\n"
        elif role == "assistant":
            prompt += f"<|assistant|>\n{msg['content']}\n"
    prompt += "<|assistant|>\n"
    return prompt

def generate_response(messages, max_new_tokens):
    prompt = format_messages(messages)
    #print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        #top_p=0.95,
        #repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("<|assistant|>")[-1]

In [98]:

# JOSHUA - NEEDS WORK, VERY SHIT
def generate_search_tag(location_tag: str, natural_query: str) -> str:
    print(f"[1] Generating search tag for input: {natural_query}")

    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant that rewrites user queries into short, search-engine-friendly tags.\n"
                "Your goal is to identify the type of business or service the user is looking for.\n\n"
                "Rules:\n"
                "- You are just returning a simple search tag\n"
                "- Use plain nouns like 'pc store', 'vegan café', 'soup restaurant', etc.\n"
                "- No punctuation, no quotes, no *, just plain text.\n"
                "- You must return a concise, single tag that captures the essence of the user's intent.\n"
            )
        },
        {
            "role": "user",
            "content": natural_query
        }
    ]

    response = generate_response(messages, max_new_tokens=20)
    tag = response.strip().split('\n')[0].strip()
    tag = re.sub(r'[^a-zA-Z0-9\s]', '', tag).strip().lower()
    tag = f"{tag} {location_tag}"
    print(f"[1.2] Generated tag: {tag}")
    return tag


# LOOK AT PARSING PURE HTML AS TEXT RATHER THAN JSON
def getCleanedText(html_text: str) -> str:
    print("[2] Cleaning HTML...")
    soup = BeautifulSoup(html_text, "html.parser")
    for tag in soup(["script", "style", "noscript", "footer", "header"]):
        tag.extract()
    text = soup.get_text(separator=" ", strip=True)
    cleaned = " ".join(text.split())
    print(f"[2.1] Cleaned text length: {len(cleaned)} chars")
    return cleaned

# COMBINE THIS WITH THE FUNCTION ABOVE OR WORK INTO MAIN LOOP
def getBusinessUrls(params: dict, searx_instance:str , urls: list[str]) -> list[dict]:
    print(f"[3] Fetching and cleaning {len(urls)} URLs")
    cleaned_pages = []
    for idx, url in enumerate(urls):
        print(f"[3.{idx}] Downloading: {url}")
        try:
            params_with_query = params.copy()
            params_with_query["q"] = url
            #response = requests.get(url, timeout=10)
            response = requests.get(f"{searx_instance}/search", params=params_with_query, timeout=10)
            response.raise_for_status()
            clean_text = getCleanedText(response.text)
            print(f"[3.{idx}.1] Page cleaned successfully ({len(clean_text)} chars)")
            cleaned_pages.append({"url": url, "text": clean_text})
        except Exception as e:
            print(f"[!] Failed to fetch {url}: {e}")
    return cleaned_pages

def relevanceCheck(cleaned_text: str, user_input: str, location: str) -> bool:
    print(f"[4] Relevance check...")
    limited_content = cleaned_text[:1000]

    messages = [
        {
            "role": "system",
            "content": (
                "You are a strict validator. Only reply with 'yes' or 'no'.\n\n"
                "Your task is to determine if the following page content is related to the user’s intent "
                "and describes a real-world business thats or organization that could be contacted.\n\n"
                "Guidelines:\n"
                "- Look for signs of actual businesses: names, product listings, services, contact info, locations.\n"
                "- Only say 'yes' to pages that clearly describe physical businesses or service providers.\n"
                "- Only say 'yes' to pages that are in the same location.\n"
                "- Do NOT say 'yes' to pages like forums, blogs, Wikipedia, or generic search results.\n"
                "- NEVER say 'yes' if the content appears to be:\n"
                "    • JSON metadata\n"
                "    • CAPTCHA pages\n"
                "    • Access denied / blocked pages\n"
                "    • Search engine error messages or redirection notices\n"
            )
        },
        {
            "role": "user",
            "content": (
                f"Intent: {user_input}\n"
                f"Location: {location}\n"
                f"Page content:\n{limited_content}\n"
                "yes or no:"
            )
        }
    ]

    response = generate_response(messages, max_new_tokens=2)
    decoded = response.lower().strip()
    print(f"[4.1] Relevance result: {decoded}")
    return decoded.startswith("yes")

def summarizeRelatedBusiness(page_text: str, user_input: str) -> str:
    print(f"[5] Summarizing page...")
    print(f"[5.1] Page Text: {page_text}")

    messages = [
        {"role": "system", "content": (
            "You are a business summarization assistant. Your job is to extract useful business information from webpages.\n"
            "Focus on identifying the **Name of business** **type of business**, **services they provide**, **industries or customers they serve**, and **location** or **email** if mentioned.\n"
            "Be concise but informative. This summary will be passed into another agent that writes cold outreach emails, so provide enough context for that.\n"
            "Avoid vague language. Only summarize if you detect an actual business presence (e.g., a service provider, store, firm, platform, etc)."
        )},
        {"role": "user", "content": (
            f"User is searching for: {user_input}\n"
            f"Here is the page content:\n"
            f"{page_text[:2000]}\n\n"
            "Summarize the business based on this content:"
        )}
    ]

    summary = generate_response(messages, max_new_tokens=400).strip()
    print(f"[5.2] Summary length: {len(summary)} chars")
    return summary


In [ ]:
def search_and_summarize(params: dict, business_query, location_tag: str, top_k: int, pages: int, clean_limit: int):
    print(f"[0] Starting search and summarize for query: {business_query}")
    search_term = generate_search_tag(location_tag, business_query)
    print(f"TEST: {search_term}")
    print(f"[0.1] Using search term: {search_term}")

    searx_instance = "http://localhost:32768"
    all_results = []

    # Collect multiple pages
    #offset = 0
    for page_num in range(pages):
        params_with_query = params.copy()
        params_with_query["q"] = search_term
        #params_with_query["start"] = 10 #offset
        params_with_query["pageno"] = page_num + 1
        #params_with_query["start"] = (page_num+1) * 10
        try:
            response = requests.get(f"{searx_instance}/search", params=params_with_query, timeout=10)
            response.raise_for_status()
            page_results = response.json().get("results", [])
            print(f"[0.{page_num}] Retrieved {len(page_results)} results from page {page_num + 1}")
            #offset += len(page_results)
            all_results.extend(page_results)
        except Exception as e:
            print(f"[!] Error retrieving page {page_num + 1}: {e}")

    # Deduplicate and extract URLs
    # for i, result in enumerate(all_results):
    #     url = result.get("url")
    #     if url:
    #         print(f"[{i}] {url}")

    urls = list({r["url"] for r in all_results if "url" in r})
    print(f"[0.2] Total unique URLs collected: {len(urls)}")

    cleaned_urls = urls if clean_limit is None else urls[:clean_limit]
    cleaned_pages = getBusinessUrls(params, searx_instance, cleaned_urls)
    summaries = []

    for i, page in enumerate(cleaned_pages):
        print(f"[6.{i}] Checking page: {page['url']}")
        if relevanceCheck(page["text"], business_query, location_tag):
            summary = summarizeRelatedBusiness(page["text"], business_query)
            summaries.append((page["url"], summary))
            print(f"[+] Relevant page added: {page['url']}")
        else:
            print(f"[-] Page deemed not relevant: {page['url']}")
        if len(summaries) >= top_k:
            print(f"[✓] Reached top_k = {top_k} summaries")
            break

    print(f"\n\n[✓] Done. Collected {len(summaries)} relevant summaries.")
    return summaries




def display_summaries(summary_results, wrap_width=80):
    df = pd.DataFrame(summary_results, columns=["URL", "Summary"])
    wrapper = textwrap.TextWrapper(width=wrap_width)
    df["Summary"] = df["Summary"].apply(lambda text: "\n".join(wrapper.wrap(text)))

    # Print nicely using tabulate
    print(tabulate(df, headers="keys", tablefmt="fancy_grid", showindex=False))

#business = "I want a business that sells pc's"
#business = "Looking for a nice restraunt the sells eggs based dishes"
business = "I am looking for a business that is also in the office supply market"
location = "Brisbane"

DEFAULT_SEARCH_PARAMS = {
    "categories": "1",
    "language": "en",
    "safesearch": "0",
    "format": "json",
    "engines": "bing,startpage,brave,mojeek,qwant,google,duckduckgo"
}

summary_results = search_and_summarize(DEFAULT_SEARCH_PARAMS, business, location, 5, 5, None)

display_summaries(summary_results, wrap_width=100)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[0] Starting search and summarize for query: I am looking for a business that is also in the office supply market
[1] Generating search tag for input: I am looking for a business that is also in the office supply market
[1.2] Generated tag: office supply store Brisbane
TEST: office supply store Brisbane
[0.1] Using search term: office supply store Brisbane
[0.0] Retrieved 10 results from page 1
[0.1] Retrieved 10 results from page 2
[0.2] Retrieved 10 results from page 3
[0.3] Retrieved 10 results from page 4
[0.4] Retrieved 10 results from page 5
[0.2] Total unique URLs collected: 50
[3] Fetching and cleaning 50 URLs
[3.0] Downloading: https://www.onet.net.au/shop/en/snows/stationery
[2] Cleaning HTML...
[2.1] Cleaned text length: 6302 chars
[3.0.1] Page cleaned successfully (6302 chars)
[3.1] Downloading: https://www.brisbaneofficesupplies.com.au/about-us
[2] Cleaning HTML...
[2.1] Cleaned text length: 6458 chars
[3.1.1] Page cleaned successfully (6458 chars)
[3.2] Downloading: https

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[2] Cleaning HTML...
[2.1] Cleaned text length: 5467 chars
[3.49.1] Page cleaned successfully (5467 chars)
[6.0] Checking page: https://www.onet.net.au/shop/en/snows/stationery
[4] Relevance check...
[4.1] Relevance result: no
[-] Page deemed not relevant: https://www.onet.net.au/shop/en/snows/stationery
[6.1] Checking page: https://www.brisbaneofficesupplies.com.au/about-us
[4] Relevance check...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

[4.1] Relevance result: no
[-] Page deemed not relevant: https://2481-au.all.biz/
[6.16] Checking page: https://www.officeworks.com.au/
[4] Relevance check...
[4.1] Relevance result: yes
[5] Summarizing page...
[5.1] Page Text: {"query": "https://www.officeworks.com.au/", "number_of_results": 0, "results": [{"url": "https://www.officeworks.com.au/", "title": "Officeworks | Stationery, Laptops, Furniture, Paper at Low Prices", "content": "Officeworks is Australia's leading online store for Office Supplies, Stationery, Office Furniture, and much more. \u00b7 Find products, brands, categories and more.", "thumbnail": null, "engine": "google", "template": "default.html", "parsed_url": ["https", "www.officeworks.com.au", "/", "", "", ""], "img_src": "", "priority": "", "engines": ["google"], "positions": [1], "score": 1.0, "category": "general"}, {"url": "https://www.officeworks.com.au/shop/officeworks/storelocator", "title": "Store Locator", "content": "Find address, trading hours, service

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[5.2] Summary length: 686 chars
[+] Relevant page added: https://www.officeworks.com.au/
[6.17] Checking page: https://www.officeworks.com.au/shop/officeworks/c/office-supplies
[4] Relevance check...
[4.1] Relevance result: no.
[-] Page deemed not relevant: https://www.officeworks.com.au/shop/officeworks/c/office-supplies
[6.18] Checking page: https://www.ony.com.au/
[4] Relevance check...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[4.1] Relevance result: yes
[5] Summarizing page...
[5.1] Page Text: {"query": "https://www.ony.com.au/", "number_of_results": 0, "results": [{"url": "https://www.ony.com.au/", "title": "Office Supplies Yatala Gold Coast | Office National", "content": "AUSTRALIA. 07 3807 4447 \u00b7 sales@ony.com.au. Monday - Friday 7:30am to 4:30pm. Closed Public Holidays. Office National Yatala is an. Australian-owned office ...", "thumbnail": null, "engine": "google", "template": "default.html", "parsed_url": ["https", "www.ony.com.au", "/", "", "", ""], "img_src": "", "priority": "", "engines": ["google"], "positions": [1], "score": 1.0, "category": "general"}, {"url": "https://www.ony.com.au/login.aspx", "title": "Login - Office National Yatala", "content": "Office National Yatala is a Queensland based Australian-owned business providing office supplies, furniture, printing and design. Payments. Card Appected.", "thumbnail": null, "engine": "google", "template": "default.html", "parsed_url": ["htt